In [1]:
#Import all required libraries
import pandas as pd
import requests
import numpy as np
from datetime import datetime
import time
import matplotlib.pyplot as plt

In [2]:
def get_data(tscurrent, network):
    """This function posts a request to make an API call to Decentraland collections Subgraph URL
    parameters:
    ------------
    tscurrent: The timestamp to use in the query. The response will be the next 1000 sales after
    this timestamp. 
    network: the string name of the network, either ethereum or matic
    return:
    -------
    response.json(): queried data in JSON format
    """

    query = """
    {
      sales(
      where: {nft_: {category: "wearable"}, timestamp_gt: """ + str(tscurrent) + """}
        first: 1000
        orderBy: timestamp
        orderDirection: asc
      ) {
        price
        royaltiesCut
        feesCollectorCut
        timestamp
        type
        nft {
          itemType
          metadata {
            wearable {
              bodyShapes
              category
              collection
              rarity
            }
          }
        }

      }
    }

    """
    #print(query)

    response = requests.post('https://api.thegraph.com/subgraphs/name/decentraland/collections-' + str(network) + '-mainnet'
                             '',
                             json={"query":query})

    if response.status_code == 200: # code 200 means no errors 
        return response.json()
    else: # if errors, print the error code for debugging
        raise Exception("Query failed with return code {}".format(response.staus_code))


In [3]:
#test cell
data = get_data(0, "ethereum")
#display(data)
#data['data']['sales'][999]['timestamp']


In [4]:
#test cell
len(data['data']['sales'])
data['data']['sales'][999]['timestamp']

'1582437758'

In [14]:
def get_all_data(tscurrent, network):    
    """This function loops through making API calls to Decentraland collections Subgraph URL
    parameters:
    ------------
    tscurrent: The timestamp to use in the query. The response will be the next 1000 sales after
    this timestamp. 
    network: the string name of the network, either ethereum or matic
    return:
    -------
    df of all the responses concatenated together
    """
    #start df and counter and timer
    dfOG = pd.DataFrame()
    counter = 0
    start = time.time()
    
    #continue looping through the data until a "break" is hit 
    while True:
        #Get the data from above function
        data = get_data(tscurrent, network)
        
        #for Matic, was hitting errors after looping 100 times, this sleep stopped the error 
        time.sleep(1)
        
        #if no data left, break out of loop
        if len(data['data']['sales']) == 0:
            print("break")
            break

        #The commented out code gave future wartning: FutureWarning: The frame.append method is deprecated 
        #and will be removed from pandas in a future version. Use pandas.concat instead.
        #dfOG = dfOG.append(pd.DataFrame.from_dict(data['data']['sales']))
        
        #Used instead of above
        #concat all previous data with the new data
        dfOG = pd.concat([dfOG,pd.DataFrame.from_dict(data['data']['sales'])], ignore_index = True)
        
        #if length is less than 1000, then this represents the last of the sales
        #break from loop
        if len(data['data']['sales']) < 1000:
            print("break")
            break
            
        #update the timestamp to the last sale retrieved, and increment counter
        tscurrent = data['data']['sales'][999]['timestamp']
        counter += 1
        
        #print current timestamp and shape for tracking
        print(tscurrent)
        print(dfOG.shape)
        #print(tscurrent)
    
    #end run time and print time
    end = time.time()
    print(end-start)
    dfOG.head()
    #print( dfOGnewmethod.head())
    
    return dfOG

In [8]:
#get data for ethereum
dfOG = get_all_data(0, "ethereum")

1
(1000, 6)
2
(2000, 6)
3
(3000, 6)
4
(4000, 6)
5
(5000, 6)
6
(6000, 6)
7
(7000, 6)
8
(8000, 6)
9
(9000, 6)
break
20.711683988571167


In [15]:
#get data for matic/Polygon
dfP = get_all_data(0, "matic")

1624741533
(1000, 6)
1626738396
(2000, 6)
1629076342
(3000, 6)
1631253852
(4000, 6)
1633221181
(5000, 6)
1634924178
(6000, 6)
1635173197
(7000, 6)
1635607399
(8000, 6)
1636208104
(9000, 6)
1637023942
(10000, 6)
1637590053
(11000, 6)
1638107394
(12000, 6)
1638597951
(13000, 6)
1639096406
(14000, 6)
1639465703
(15000, 6)
1639713398
(16000, 6)
1639969668
(17000, 6)
1640195954
(18000, 6)
1640337769
(19000, 6)
1640504200
(20000, 6)
1640748109
(21000, 6)
1640892310
(22000, 6)
1641051931
(23000, 6)
1641262865
(24000, 6)
1641518097
(25000, 6)
1641599788
(26000, 6)
1641795533
(27000, 6)
1641957858
(28000, 6)
1642118695
(29000, 6)
1642155217
(30000, 6)
1642213220
(31000, 6)
1642267154
(32000, 6)
1642344879
(33000, 6)
1642466502
(34000, 6)
1642609137
(35000, 6)
1642644162
(36000, 6)
1642646488
(37000, 6)
1642649120
(38000, 6)
1642716718
(39000, 6)
1642865333
(40000, 6)
1642989141
(41000, 6)
1643188829
(42000, 6)
1643353364
(43000, 6)
1643482151
(44000, 6)
1643660258
(45000, 6)
1643826189
(46000, 

In [16]:
#take a look at data
dfP

,price,royaltiesCut,feesCollectorCut,timestamp,type,nft
0,500000000000000000000,0,12500000000000000000,1623352144,order,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
1,100000000000000000000,0,2500000000000000000,1623352460,order,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
2,100000000000000000000,0,2500000000000000000,1623352528,order,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
3,100000000000000000000,0,2500000000000000000,1623352602,order,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
4,100000000000000000000,0,2500000000000000000,1623353328,order,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
...,...,...,...,...,...,...
182971,15000000000000000000,0,375000000000000000,1681825699,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
182972,1000000000000000000,0,25000000000000000,1681826803,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
182973,1500000000000000000,0,37500000000000000,1681826847,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
182974,3000000000000000000,0,75000000000000000,1681832356,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."


In [17]:
#take a look at data
dfOG

,price,royaltiesCut,feesCollectorCut,timestamp,type,nft
0,11111000000000000000000,0,111110000000000000000,1581364178,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
1,25000000000000000000000,0,250000000000000000000,1581364712,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
2,150000000000000000000,0,3750000000000000000,1581960831,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
3,155000000000000000000,0,3875000000000000000,1581960870,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
4,180000000000000000000,0,4500000000000000000,1581963425,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
...,...,...,...,...,...,...
9277,100000000000000000000,0,2500000000000000000,1674190271,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
9278,149000000000000000000,0,3725000000000000000,1674847043,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
9279,30000000000000000000,0,750000000000000000,1676773607,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
9280,30000000000000000000,0,750000000000000000,1678262027,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."


In [26]:
#concat the ethereum and matic df together
df = pd.concat([dfOG,dfP], ignore_index = True)
df

,price,royaltiesCut,feesCollectorCut,timestamp,type,nft
0,11111000000000000000000,0,111110000000000000000,1581364178,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
1,25000000000000000000000,0,250000000000000000000,1581364712,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
2,150000000000000000000,0,3750000000000000000,1581960831,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
3,155000000000000000000,0,3875000000000000000,1581960870,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
4,180000000000000000000,0,4500000000000000000,1581963425,order,"{'itemType': 'wearable_v1', 'metadata': {'wear..."
...,...,...,...,...,...,...
192253,15000000000000000000,0,375000000000000000,1681825699,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
192254,1000000000000000000,0,25000000000000000,1681826803,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
192255,1500000000000000000,0,37500000000000000,1681826847,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."
192256,3000000000000000000,0,75000000000000000,1681832356,mint,"{'itemType': 'wearable_v2', 'metadata': {'wear..."


In [27]:
#a few columns have multiple items in it
#split them a part and concat together
df = pd.concat([df.drop(['nft'], axis = 1), df['nft'].apply(pd.Series)],axis = 1)
df = pd.concat([df.drop(['metadata'], axis = 1), df['metadata'].apply(pd.Series)],axis = 1)           
df = pd.concat([df.drop(['wearable'], axis = 1), df['wearable'].apply(pd.Series)],axis = 1)
df


,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity
0,11111000000000000000000,0,111110000000000000000,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic
1,25000000000000000000000,0,250000000000000000000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary
2,150000000000000000000,0,3750000000000000000,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare
3,155000000000000000000,0,3875000000000000000,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare
4,180000000000000000000,0,4500000000000000000,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare
...,...,...,...,...,...,...,...,...,...,...
192253,15000000000000000000,0,375000000000000000,1681825699,mint,wearable_v2,[BaseFemale],upper_body,0x9376f00e31c46a05dcad41ef7308a25ff1ccffb0,legendary
192254,1000000000000000000,0,25000000000000000,1681826803,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0x0a04cca75b74c69ac5ad2e219d1b9c180d8fc746,common
192255,1500000000000000000,0,37500000000000000,1681826847,mint,wearable_v2,"[BaseMale, BaseFemale]",eyewear,0x8cdf396bc0244544af18e30bf4f42db50d75b030,epic
192256,3000000000000000000,0,75000000000000000,1681832356,mint,wearable_v2,"[BaseMale, BaseFemale]",mouth,0x563e2081b3cd716ed76fc0993b7e49939cb342a5,epic


In [28]:
#see some df info
df.describe()

,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity
count,192258,192258,192258,192258,192258,192258,192258,192258,192258,192258
unique,1154,690,786,182551,3,3,4,16,3322,7
top,1000000000000000000,0,0,1668355148,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0xca3f06d124850ea57db0d6fcc4295b2e03a2bbe3,epic
freq,26691,148883,43375,22,128909,182082,164068,59893,7832,54053


In [29]:
#Change all prices to float and get it to correct units
#the prices are recorded with 18 extra 0s
df['price'] = df['price'].astype(np.float64)
df['price'] = df['price']/10**18
df['royaltiesCut'] = df['royaltiesCut'].astype(np.float64)
df['royaltiesCut'] = df['royaltiesCut']/10**18
df['feesCollectorCut'] = df['feesCollectorCut'].astype(np.float64)
df['feesCollectorCut'] = df['feesCollectorCut']/10**18

#Change to correct types
df['timestamp'].astype(int)
df['Date'] = pd.to_datetime(df['timestamp'],unit='s').dt.date
print(df.shape)
df.head()

(192258, 11)


,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity,Date
0,11111.0,0.0,111.110,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic,2020-02-10
1,25000.0,0.0,250.000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary,2020-02-10
2,150.0,0.0,3.750,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17
3,155.0,0.0,3.875,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17
4,180.0,0.0,4.500,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17


In [34]:
#check the counts of same variables
print(df['itemType'].value_counts())
print(df['type'].value_counts())

wearable_v2          182082
wearable_v1            9282
smart_wearable_v1       894
Name: itemType, dtype: int64
mint     128909
order     57107
bid        6242
Name: type, dtype: int64


In [35]:
#merge royaltiescut and feescollectors cut to one column
mask = df['feesCollectorCut'] == 0
df['Fee'] = np.where(mask, df['royaltiesCut'], df['feesCollectorCut'])
df

,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity,Date,Fee
0,11111.00,0.00000,111.1100,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic,2020-02-10,111.11000
1,25000.00,0.00000,250.0000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary,2020-02-10,250.00000
2,150.00,0.00000,3.7500,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.75000
3,155.00,0.00000,3.8750,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.87500
4,180.00,0.00000,4.5000,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,4.50000
...,...,...,...,...,...,...,...,...,...,...,...,...
192253,15.00,0.00000,0.3750,1681825699,mint,wearable_v2,[BaseFemale],upper_body,0x9376f00e31c46a05dcad41ef7308a25ff1ccffb0,legendary,2023-04-18,0.37500
192254,1.00,0.00000,0.0250,1681826803,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0x0a04cca75b74c69ac5ad2e219d1b9c180d8fc746,common,2023-04-18,0.02500
192255,1.50,0.00000,0.0375,1681826847,mint,wearable_v2,"[BaseMale, BaseFemale]",eyewear,0x8cdf396bc0244544af18e30bf4f42db50d75b030,epic,2023-04-18,0.03750
192256,3.00,0.00000,0.0750,1681832356,mint,wearable_v2,"[BaseMale, BaseFemale]",mouth,0x563e2081b3cd716ed76fc0993b7e49939cb342a5,epic,2023-04-18,0.07500


In [36]:
#wearable_v1 is Ethereum, the other two are Polygon
#make column to say which network
mask1 = df['itemType'] == "wearable_v1"
df['Network'] = np.where(mask1, "Ethereum", "Polygon")
df

,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity,Date,Fee,Network
0,11111.00,0.00000,111.1100,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic,2020-02-10,111.11000,Ethereum
1,25000.00,0.00000,250.0000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary,2020-02-10,250.00000,Ethereum
2,150.00,0.00000,3.7500,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.75000,Ethereum
3,155.00,0.00000,3.8750,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.87500,Ethereum
4,180.00,0.00000,4.5000,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,4.50000,Ethereum
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192253,15.00,0.00000,0.3750,1681825699,mint,wearable_v2,[BaseFemale],upper_body,0x9376f00e31c46a05dcad41ef7308a25ff1ccffb0,legendary,2023-04-18,0.37500,Polygon
192254,1.00,0.00000,0.0250,1681826803,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0x0a04cca75b74c69ac5ad2e219d1b9c180d8fc746,common,2023-04-18,0.02500,Polygon
192255,1.50,0.00000,0.0375,1681826847,mint,wearable_v2,"[BaseMale, BaseFemale]",eyewear,0x8cdf396bc0244544af18e30bf4f42db50d75b030,epic,2023-04-18,0.03750,Polygon
192256,3.00,0.00000,0.0750,1681832356,mint,wearable_v2,"[BaseMale, BaseFemale]",mouth,0x563e2081b3cd716ed76fc0993b7e49939cb342a5,epic,2023-04-18,0.07500,Polygon


In [39]:
def Body(row):
    """This function returns a single letter for bodyshape.
    ------------
    row: This function will be used in a df.apply and each row of the df will sent through
    this function
    return:
    -------
    M for male, F for female, U for Unisex
    """
    if row['bodyShapes'] == ['BaseMale']:
        return 'M'
    elif row['bodyShapes'] == ['BaseFemale']:
        return 'F'
    elif row['bodyShapes'] == ['BaseMale','BaseFemale']:
        return 'U'
    elif row['bodyShapes'] == ['BaseFemale','BaseMale']:
        return 'U'


In [40]:
#apply above function to df
df['BodyShape'] = df.apply(Body, axis=1)
df

,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity,Date,Fee,Network,BodyShape
0,11111.00,0.00000,111.1100,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic,2020-02-10,111.11000,Ethereum,M
1,25000.00,0.00000,250.0000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary,2020-02-10,250.00000,Ethereum,U
2,150.00,0.00000,3.7500,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.75000,Ethereum,U
3,155.00,0.00000,3.8750,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.87500,Ethereum,U
4,180.00,0.00000,4.5000,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,4.50000,Ethereum,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192253,15.00,0.00000,0.3750,1681825699,mint,wearable_v2,[BaseFemale],upper_body,0x9376f00e31c46a05dcad41ef7308a25ff1ccffb0,legendary,2023-04-18,0.37500,Polygon,F
192254,1.00,0.00000,0.0250,1681826803,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0x0a04cca75b74c69ac5ad2e219d1b9c180d8fc746,common,2023-04-18,0.02500,Polygon,U
192255,1.50,0.00000,0.0375,1681826847,mint,wearable_v2,"[BaseMale, BaseFemale]",eyewear,0x8cdf396bc0244544af18e30bf4f42db50d75b030,epic,2023-04-18,0.03750,Polygon,U
192256,3.00,0.00000,0.0750,1681832356,mint,wearable_v2,"[BaseMale, BaseFemale]",mouth,0x563e2081b3cd716ed76fc0993b7e49939cb342a5,epic,2023-04-18,0.07500,Polygon,U


In [41]:
#change type
df["Date"] = df["Date"].astype('datetime64[ns]')

In [42]:
df.dtypes

price                      float64
royaltiesCut               float64
feesCollectorCut           float64
timestamp                   object
type                        object
itemType                    object
bodyShapes                  object
category                    object
collection                  object
rarity                      object
Date                datetime64[ns]
Fee                        float64
Network                     object
BodyShape                   object
dtype: object

In [43]:
#Get the first date of a sale from the data
firstdate = df.loc[0,'Date']
lastdate = df.loc[len(df['Date'])-1,'Date']

#Get the quantity of days from first sale to today
#This will be used in upcoming cells to get daily historical coin prices
days = datetime.now() - firstdate
days.days

1163

In [44]:
#api url for MANA historical prices
url = 'https://api.coingecko.com/api/v3/coins/decentraland/market_chart?vs_currency=usd&days=' + str(days.days+1)

#make API call to coingecko
response = requests.get(url)
#response.json()['prices']

#set json to df and set columns to correct type
manadf = pd.DataFrame(response.json()['prices'], columns = ['timestamp','MANA price (USD)'])
manadf['Date'] = pd.to_datetime(manadf['timestamp'],unit='ms').dt.date
manadf["Date"] = manadf["Date"].astype('datetime64[ns]')

#drop last row because both same day
manadf.drop(manadf.tail(1).index,inplace=True)

#drop timestamp, not needed
manadf = manadf.drop(['timestamp'], axis=1)
manadf

,MANA price (USD),Date
0,0.064333,2020-02-10
1,0.060102,2020-02-11
2,0.064091,2020-02-12
3,0.061613,2020-02-13
4,0.060294,2020-02-14
...,...,...
1159,0.618217,2023-04-14
1160,0.640651,2023-04-15
1161,0.640651,2023-04-16
1162,0.644682,2023-04-17


In [46]:
#api url for ETH historical prices
url = 'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days=' + str(days.days+1)

#make API call to coingecko
response = requests.get(url)

#set json to df and set columns to correct type
ethdf = pd.DataFrame(response.json()['prices'], columns = ['timestamp','ETH price (USD)'])
ethdf['Date'] = pd.to_datetime(ethdf['timestamp'],unit='ms').dt.date
ethdf["Date"] = ethdf["Date"].astype('datetime64[ns]')

#drop last row because both same day
ethdf.drop(ethdf.tail(1).index,inplace=True)

#drop timestamp, not needed
ethdf = ethdf.drop(['timestamp'], axis=1)
ethdf

,ETH price (USD),Date
0,228.292261,2020-02-10
1,224.146997,2020-02-11
2,236.785349,2020-02-12
3,264.032768,2020-02-13
4,267.670445,2020-02-14
...,...,...
1159,2012.785415,2023-04-14
1160,2102.946188,2023-04-15
1161,2093.166331,2023-04-16
1162,2118.598789,2023-04-17


In [47]:
#Merge main df with MANA and ETH historical prices
df = pd.merge(df,manadf,on='Date', how='left')

In [49]:
df = pd.merge(df,ethdf,on='Date', how='left')
df

,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity,Date,Fee,Network,BodyShape,MANA price (USD),ETH price (USD)
0,11111.00,0.00000,111.1100,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic,2020-02-10,111.11000,Ethereum,M,0.064333,228.292261
1,25000.00,0.00000,250.0000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary,2020-02-10,250.00000,Ethereum,U,0.064333,228.292261
2,150.00,0.00000,3.7500,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.75000,Ethereum,U,0.057185,262.156244
3,155.00,0.00000,3.8750,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.87500,Ethereum,U,0.057185,262.156244
4,180.00,0.00000,4.5000,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,4.50000,Ethereum,U,0.057185,262.156244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192253,15.00,0.00000,0.3750,1681825699,mint,wearable_v2,[BaseFemale],upper_body,0x9376f00e31c46a05dcad41ef7308a25ff1ccffb0,legendary,2023-04-18,0.37500,Polygon,F,0.691581,2077.535665
192254,1.00,0.00000,0.0250,1681826803,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0x0a04cca75b74c69ac5ad2e219d1b9c180d8fc746,common,2023-04-18,0.02500,Polygon,U,0.691581,2077.535665
192255,1.50,0.00000,0.0375,1681826847,mint,wearable_v2,"[BaseMale, BaseFemale]",eyewear,0x8cdf396bc0244544af18e30bf4f42db50d75b030,epic,2023-04-18,0.03750,Polygon,U,0.691581,2077.535665
192256,3.00,0.00000,0.0750,1681832356,mint,wearable_v2,"[BaseMale, BaseFemale]",mouth,0x563e2081b3cd716ed76fc0993b7e49939cb342a5,epic,2023-04-18,0.07500,Polygon,U,0.691581,2077.535665


In [50]:
#calculate fee percentages
df['Fee Percentage'] = df['Fee']/df['price']
df

,price,royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity,Date,Fee,Network,BodyShape,MANA price (USD),ETH price (USD),Fee Percentage
0,11111.00,0.00000,111.1100,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic,2020-02-10,111.11000,Ethereum,M,0.064333,228.292261,0.010
1,25000.00,0.00000,250.0000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary,2020-02-10,250.00000,Ethereum,U,0.064333,228.292261,0.010
2,150.00,0.00000,3.7500,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.75000,Ethereum,U,0.057185,262.156244,0.025
3,155.00,0.00000,3.8750,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.87500,Ethereum,U,0.057185,262.156244,0.025
4,180.00,0.00000,4.5000,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,4.50000,Ethereum,U,0.057185,262.156244,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192253,15.00,0.00000,0.3750,1681825699,mint,wearable_v2,[BaseFemale],upper_body,0x9376f00e31c46a05dcad41ef7308a25ff1ccffb0,legendary,2023-04-18,0.37500,Polygon,F,0.691581,2077.535665,0.025
192254,1.00,0.00000,0.0250,1681826803,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0x0a04cca75b74c69ac5ad2e219d1b9c180d8fc746,common,2023-04-18,0.02500,Polygon,U,0.691581,2077.535665,0.025
192255,1.50,0.00000,0.0375,1681826847,mint,wearable_v2,"[BaseMale, BaseFemale]",eyewear,0x8cdf396bc0244544af18e30bf4f42db50d75b030,epic,2023-04-18,0.03750,Polygon,U,0.691581,2077.535665,0.025
192256,3.00,0.00000,0.0750,1681832356,mint,wearable_v2,"[BaseMale, BaseFemale]",mouth,0x563e2081b3cd716ed76fc0993b7e49939cb342a5,epic,2023-04-18,0.07500,Polygon,U,0.691581,2077.535665,0.025


In [51]:
#check value counts for fee percentages
#almost all are the same, so not needed
print(df['Fee Percentage'].value_counts())

0.025    139087
0.025     51800
0.025      1334
0.025        35
0.010         1
0.010         1
Name: Fee Percentage, dtype: int64


In [52]:
#Rename columns to clarify
df.rename(columns = {'price':'sale price (MANA)'}, inplace = True)

#Get sale price in USD
df['sale price (USD)'] = df['sale price (MANA)']*df['MANA price (USD)']
df

,sale price (MANA),royaltiesCut,feesCollectorCut,timestamp,type,itemType,bodyShapes,category,collection,rarity,Date,Fee,Network,BodyShape,MANA price (USD),ETH price (USD),Fee Percentage,sale price (USD)
0,11111.00,0.00000,111.1100,1581364178,order,wearable_v1,[BaseMale],upper_body,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,epic,2020-02-10,111.11000,Ethereum,M,0.064333,228.292261,0.010,714.798988
1,25000.00,0.00000,250.0000,1581364712,order,wearable_v1,"[BaseMale, BaseFemale]",mask,0xc04528c14c8ffd84c7c1fb6719b4a89853035cdd,legendary,2020-02-10,250.00000,Ethereum,U,0.064333,228.292261,0.010,1608.313807
2,150.00,0.00000,3.7500,1581960831,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.75000,Ethereum,U,0.057185,262.156244,0.025,8.577801
3,155.00,0.00000,3.8750,1581960870,order,wearable_v1,"[BaseMale, BaseFemale]",helmet,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,3.87500,Ethereum,U,0.057185,262.156244,0.025,8.863727
4,180.00,0.00000,4.5000,1581963425,order,wearable_v1,"[BaseMale, BaseFemale]",hat,0xc1f4b0eea2bd6690930e6c66efd3e197d620b9c2,rare,2020-02-17,4.50000,Ethereum,U,0.057185,262.156244,0.025,10.293361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192253,15.00,0.00000,0.3750,1681825699,mint,wearable_v2,[BaseFemale],upper_body,0x9376f00e31c46a05dcad41ef7308a25ff1ccffb0,legendary,2023-04-18,0.37500,Polygon,F,0.691581,2077.535665,0.025,10.373708
192254,1.00,0.00000,0.0250,1681826803,mint,wearable_v2,"[BaseMale, BaseFemale]",upper_body,0x0a04cca75b74c69ac5ad2e219d1b9c180d8fc746,common,2023-04-18,0.02500,Polygon,U,0.691581,2077.535665,0.025,0.691581
192255,1.50,0.00000,0.0375,1681826847,mint,wearable_v2,"[BaseMale, BaseFemale]",eyewear,0x8cdf396bc0244544af18e30bf4f42db50d75b030,epic,2023-04-18,0.03750,Polygon,U,0.691581,2077.535665,0.025,1.037371
192256,3.00,0.00000,0.0750,1681832356,mint,wearable_v2,"[BaseMale, BaseFemale]",mouth,0x563e2081b3cd716ed76fc0993b7e49939cb342a5,epic,2023-04-18,0.07500,Polygon,U,0.691581,2077.535665,0.025,2.074742


In [53]:
#check correlations between variables
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

C:\Users\Bamar\AppData\Local\Temp\ipykernel_23784\3784545759.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = df.corr()


,sale price (MANA),royaltiesCut,feesCollectorCut,Fee,MANA price (USD),ETH price (USD),Fee Percentage,sale price (USD)
sale price (MANA),1.000000,0.166592,0.977453,0.992059,-0.101561,-0.105718,-0.196056,0.386172
royaltiesCut,0.166592,1.000000,-0.003094,0.169785,-0.014715,-0.013875,0.000104,0.236465
feesCollectorCut,0.977453,-0.003094,1.000000,0.984951,-0.101825,-0.106019,-0.080919,0.355206
Fee,0.992059,0.169785,0.984951,1.000000,-0.102890,-0.106879,-0.079727,0.390919
MANA price (USD),-0.101561,-0.014715,-0.101825,-0.102890,1.000000,0.856573,0.005078,0.040340
ETH price (USD),-0.105718,-0.013875,-0.106019,-0.106879,0.856573,1.000000,0.007135,0.080171
Fee Percentage,-0.196056,0.000104,-0.080919,-0.079727,0.005078,0.007135,1.000000,-0.018995
sale price (USD),0.386172,0.236465,0.355206,0.390919,0.040340,0.080171,-0.018995,1.000000


In [54]:
#save df to csv for use in another notebook
df.to_csv('DecentralandDataWearablesWithPolygon.csv')